# Soil moisture memory tutorial 2
### Sanjiv Kumar, Auburn University (szk0139@auburn.edu), 05/17/2023

In [67]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr

In [68]:
ds1 = xr.open_dataset("/glade/derecho/scratch/maruf/sm_data_comparison/gswp3_ensembles/sm_std_anom_ts_monthly_1971_2014_gswp_ensemble10.nc", engine="netcdf4")

In [69]:
ds1

<xarray.Dataset>
Dimensions:      (time: 528, lat: 192, lon: 288)
Coordinates:
  * time         (time) object 1971-02-01 00:00:00 ... 2015-01-01 00:00:00
  * lat          (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon          (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    sm1_anom     (time, lat, lon) float32 ...
    sm2_anom     (time, lat, lon) float32 ...
    pcp_anom     (time, lat, lon) float32 ...
    sm1_anom_dt  (time, lat, lon) float32 ...
    sm2_anom_dt  (time, lat, lon) float32 ...
    pcp_anom_dt  (time, lat, lon) float32 ...

In [70]:
sm_rz_3d = ds1.sm1_anom
sm_dl_3d = ds1.sm2_anom
P_atm_3d = ds1.pcp_anom
sm_rz_3d

<xarray.DataArray 'sm1_anom' (time: 528, lat: 192, lon: 288)>
[29196288 values with dtype=float32]
Coordinates:
  * time     (time) object 1971-02-01 00:00:00 ... 2015-01-01 00:00:00
  * lat      (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    average_op_ncl:  dim_avg_wgt_n over dimension(s): levsoi
    long_name:       volumetric soil water (vegetated landunits only)
    units:           mm3/mm3
    cell_methods:    time: mean

In [71]:
sm_memory = sm_rz_3d[0:12, :, :]
sm_memory

<xarray.DataArray 'sm1_anom' (time: 12, lat: 192, lon: 288)>
[663552 values with dtype=float32]
Coordinates:
  * time     (time) object 1971-02-01 00:00:00 ... 1972-01-01 00:00:00
  * lat      (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    average_op_ncl:  dim_avg_wgt_n over dimension(s): levsoi
    long_name:       volumetric soil water (vegetated landunits only)
    units:           mm3/mm3
    cell_methods:    time: mean

## usefule reference
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.acf.html 

In [72]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [61]:
import math
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.tsa.stattools as ST
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA

In [62]:
lat = ds1.lat
lon = ds1.lon

In [63]:
def CAL_MEM(x, s):
    # Loop to compute the autcorrelation for n lags
    n_lag = 18
    n_year = int(len(x)/12) -2
    dt_x = np.empty(n_year-1, dtype=float)
    dt_y = np.empty(n_year-1, dtype=float)
    dt_corel_sm = np.empty([n_lag+1, 2], dtype=float) 
    sm_rz_mav = x
    i = 0
    while i <= n_lag:
        dt_x = sm_rz_mav[s:len(x):12][0:n_year-1]              # Here 1 indicate the FMA season, 
        dt_y = sm_rz_mav[s+i:len(x):12][0:n_year-1]
        dt_corel_sm[i, 0] = stats.pearsonr(dt_x, dt_y)[0]
        dt_corel_sm[i, 1] = stats.pearsonr(dt_x, dt_y)[1]
        i +=1
        
    # Loop to compute the expotential decay function at finer time setp (n/10)
    
    mem_temp = np.empty([n_lag*10, 2], dtype=float)
    x_temp = range(0, n_lag+1, 1)
    diff_temp = x_temp

    k = 1
    while k <=int(n_lag*10):
        memo = 0.1*k
        y_temp = np.empty([n_lag+1], dtype=float)
        for j in x_temp:
            y_temp[j] = math.exp(-1*x_temp[j]/memo)
        
        diff_temp = (y_temp - dt_corel_sm[:, 0])**2.0
        mem_temp[k-1, 0] = memo
        mem_temp[k-1, 1] = sum(diff_temp)
        k += 1
    
    # Now finally detemining the memory as the the lag at which rmse is minimum
    rmse_min = mem_temp[0, 1]
    memory = mem_temp[0, 0]
    for j in range(1, int(n_lag*10), 1):
        if mem_temp[j, 1] < rmse_min:
            rmse_min = mem_temp[j, 1]
            memory = mem_temp[j, 0]
    return np.round(memory, 2)

In [64]:
import statsmodels.api as sm

In [65]:
for i_lat in range(len(lat)):
    for j_lon in range(len(lon)):
        sm_rz = sm_rz_3d[:, i_lat, j_lon]
        #nmc = np.count_nonzero(~np.isnan(sm_rz))
        nmc = np.count_nonzero(~np.isnan(sm_rz))
        np.isnan(sm_rz).any()  # Check for NaN values
        np.isinf(sm_rz).any()  # Check for inf values
        sm_rz = sm_rz[~np.isnan(sm_rz) & ~np.isinf(sm_rz)]
        if nmc > 100:
            for i_mon in range(12):
                temp11 = CAL_MEM(sm_rz, i_mon)
                sm_memory[i_mon, i_lat, j_lon] = temp11

/glade/work/maruf/miniconda3/envs/xarray-tutorial/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/glade/work/maruf/miniconda3/envs/xarray-tutorial/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/glade/work/maruf/miniconda3/envs/xarray-tutorial/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/glade/work/maruf/miniconda3/envs/xarray-tutorial/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRCons

In [66]:
sm_memory.to_netcdf('ensemble10_gswp3_rz.nc')